# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
all_experience_types = fct_guest_spending['park_experience_type'].drop_duplicates()

july_data = fct_guest_spending[
    (fct_guest_spending['visit_date'] >= '2024-07-01') &
    (fct_guest_spending['visit_date'] < '2024-08-01')
]

visit_spend = july_data.groupby(
    ['guest_id', 'visit_date', 'park_experience_type'], as_index=False
)['amount_spent'].sum()

avg_spending = visit_spend.groupby('park_experience_type', as_index=False)['amount_spent'].mean()
avg_spending.rename(columns={'amount_spent': 'avg_spending'}, inplace=True)

final_result = pd.DataFrame({'park_experience_type': all_experience_types})
final_result = final_result.merge(avg_spending, on='park_experience_type', how='left')
final_result['avg_spending'] = final_result['avg_spending'].fillna(0.0)

print(final_result.sort_values('park_experience_type').reset_index(drop=True))

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
august_data = fct_guest_spending[
    (fct_guest_spending['visit_date'] >= '2024-08-01') &
    (fct_guest_spending['visit_date'] < '2024-09-01')
]

guest_visit_spend = august_data.groupby(['guest_id', 'visit_date'], as_index=False)['amount_spent'].sum()

visit_counts = guest_visit_spend['guest_id'].value_counts()
multi_visit_guests = visit_counts[visit_counts > 1].index

multi_visit_data = guest_visit_spend[guest_visit_spend['guest_id'].isin(multi_visit_guests)]

first_last_visits = multi_visit_data.sort_values(['guest_id', 'visit_date']).groupby('guest_id').agg(
    first_visit_spend=('amount_spent', lambda x: x.iloc[0]),
    last_visit_spend=('amount_spent', lambda x: x.iloc[-1])
).reset_index()

first_last_visits['spending_difference'] = first_last_visits['last_visit_spend'] - first_last_visits['first_visit_spend']

print(first_last_visits.sort_values('guest_id').reset_index(drop=True))

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
sept_data = fct_guest_spending[
    (fct_guest_spending['visit_date'] >= '2024-09-01') &
    (fct_guest_spending['visit_date'] < '2024-10-01')
]

guest_spend = sept_data.groupby('guest_id', as_index=False)['amount_spent'].sum()

guest_spend = guest_spend[guest_spend['amount_spent'] > 0]

def categorize(spend):
    if spend < 50:
        return 'Low'
    elif spend < 100:
        return 'Medium'
    else:
        return 'High'

guest_spend['spending_segment'] = guest_spend['amount_spent'].apply(categorize)

print(guest_spend.sort_values(['spending_segment', 'amount_spent']).reset_index(drop=True))

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)